In [3]:
import pandas as pd
from smartsheet_dataframe import get_as_df
import os

In [4]:
# Storing Smartsheet information in variables
shitz_token = "wMvGniITjDFd7ClZPE44wtJGvZkM7Hg7mP7if" #Bily's personal token to access G-A smartsheet data. Anyone can obtain their own token
fhaid = 4523924143794052 # The ID number that references the FHA sheet
mssid = 3005704744265604 # The ID number that references the MSS sheet

# Creating the FHA and MSS dataframes
fha_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=fhaid)

mss_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=mssid)

war_df = mss_df.loc[mss_df['UniqueID'].str.contains("WAR")].copy()

In [5]:
war_inprog = war_df[war_df['Status'].isin(['IN PROCESS', 'OPEN', 'POST', 'PENDING'])].copy()
war_inprog = war_inprog[['UniqueID', 'TicketID', 'Status', 'Reporting Period', 'Current RP Expected GT', 'Future GT', 'Total GT']]
len(war_inprog['TicketID'].unique())

70

In [6]:
rp9_expected = pd.read_excel(r'C:\Users\bbrown\Documents\python_scripts\db-build-trial\WAR_RP9_InProcess\Excel_sheets\rp9_investigate_matchesRob.xlsx', sheet_name='Sheet2')

In [7]:
trial_df = pd.merge(war_inprog, rp9_expected, on='TicketID', how='left')

In [8]:
trial_df

,UniqueID,TicketID,Status,Reporting Period,Current RP Expected GT,Future GT,Total GT,RP9 Expected
0,WAR241025TT01,WAR241025TT01,PENDING,9.0,4528.0,0.0,4528.0,4528.00
1,WAR241025HD17,WAR241025HD17,OPEN,9.0,4689.54,0.0,4690.0,4690.00
2,WAR241025HD16,WAR241025HD16,OPEN,9.0,5174.26,0.0,5180.0,5180.00
3,WAR241025HD15,WAR241025HDXX,OPEN,9.0,350.58,0.0,350.58,21772.72
4,WAR241025HD14,WAR241025HDXX,OPEN,9.0,4417.97,0.0,4417.97,21772.72
...,...,...,...,...,...,...,...,...
121,WAR230303TH01,WAR230303TH01,OPEN,8.0,4253.0,,4253.0,4253.00
122,WAR230203TT01,WAR230203TTXX,IN PROCESS,8.0,1104.0,,,0.00
123,WAR230203TT02,WAR230203TTXX,IN PROCESS,8.0,3082.0,,,0.00
124,WAR230113TD04,WAR230113TD04,IN PROCESS,8.0,12680.0,,12680.0,680.25


In [9]:
trial_df.to_excel('left_look.xlsx')

In [12]:
ticket_expected = trial_df.groupby('TicketID')['RP9 Expected'].first()


In [14]:
trial_df[trial_df['Status'] == 'PENDING']

,UniqueID,TicketID,Status,Reporting Period,Current RP Expected GT,Future GT,Total GT,RP9 Expected
0,WAR241025TT01,WAR241025TT01,PENDING,9.0,4528.0,0.0,4528.0,4528.0


In [15]:
trial_df.groupby('TicketID')['RP9 Expected'].first().sum() - 4528

411316.7516191328

In [17]:
len(trial_df.groupby('TicketID')['RP9 Expected'].first())

70

In [18]:
len(trial_df)

126

In [19]:
126-70


56

In [21]:
war_rp9_expected = trial_df.groupby('TicketID')['RP9 Expected'].first().reset_index()
war_rp9_expected

,TicketID,RP9 Expected
0,WAR230113TD03,7322.00
1,WAR230113TD04,680.25
2,WAR230203TTXX,0.00
3,WAR230303TH01,4253.00
4,WAR230318CC05,0.00
...,...,...
65,WAR241025HD09,4781.69
66,WAR241025HD16,5180.00
67,WAR241025HD17,4690.00
68,WAR241025HDXX,21772.72


In [22]:
# NExt steps : tie in the rest of the MSS info that I need to populate Rob's table

In [40]:
rp9_accounting = pd.merge(war_rp9_expected, war_df[war_df['Status'].isin(['IN PROCESS', 'OPEN', 'PENDING', 'POST'])], on='TicketID', how='left')
rp9_accounting = rp9_accounting[['UniqueID', 'TicketID', 'Tract', 'CMP', 'Stand', 'County', 'Acres', 'RP9 Expected', 'Status', 'Guidance']]

In [41]:
rp9_accounting_cleaning = rp9_accounting.groupby('TicketID').agg({
    'Tract': 'first',
    'CMP': 'first',
    'Stand': 'first',
    'County': ', '.join,
    'Acres': 'sum',
    'RP9 Expected': 'first',
    'Status': ', '.join,
    'Guidance': ', '.join
}).reset_index()

In [42]:
rp9_accounting_cleaning.to_excel('rp9_filecheck.xlsx')

In [46]:
ticket_id_lookup = trial_df[['UniqueID', 'TicketID']].copy()
ticket_id_lookup.to_excel('war_ticketid_lookup.xlsx')

In [47]:
rp9_accounting_cleaning

,TicketID,Tract,CMP,Stand,County,Acres,RP9 Expected,Status,Guidance
0,WAR230113TD03,13180.0,NP,"13, 18, 23, &27",Troup,40.38,7322.00,OPEN,2023-01-26
1,WAR230113TD04,33094A & 13180,NP,1 & 2,Troup,77.71,680.25,IN PROCESS,2023-01-26
2,WAR230203TTXX,13206.0,024 018,13.0,"Meriwether, Meriwether",91.0,0.00,"IN PROCESS, IN PROCESS",","
3,WAR230303TH01,Clinch County,1.0,2.0,Clinch,121.51,4253.00,OPEN,2023-03-10
4,WAR230318CC05,BW55,0040 0009,CC1,Troup,38.0,0.00,IN PROCESS,2023-06-24
...,...,...,...,...,...,...,...,...,...
65,WAR241025HD09,Clinch,3,10,Clinch,68.0,4781.69,OPEN,
66,WAR241025HD16,Clinch,7,5,Clinch,74.0,5180.00,OPEN,
67,WAR241025HD17,Clinch,7,6,Clinch,67.0,4690.00,OPEN,
68,WAR241025HDXX,Clinch,6,5,"Clinch, Clinch, Clinch, Clinch, Clinch, Clinch",310.02,21772.72,"OPEN, OPEN, OPEN, OPEN, OPEN, OPEN",", , , , ,"


In [49]:
import sqlite3

In [50]:
conn = sqlite3.connect('./WAR_RP9_InProcess/trial-db/Harvest.db')
c = conn.cursor()


In [51]:
ticketid_table = pd.read_sql('SELECT * FROM ticketid',conn)

In [52]:
ticketid_table

,ticketid,id
0,WAR230113TD03,0
1,WAR230113TD04,1
2,WAR230203TTXX,2
3,WAR230303TH01,3
4,WAR230318CC05,4
...,...,...
201,WAR240606TH03,201
202,WAR240606TH04,202
203,WAR240719CC01,203
204,WAR240719CC02,204


In [55]:
rp9_accounting_ids = rp9_accounting_cleaning['TicketID'].unique().tolist()

ticketid_table[ticketid_table['ticketid'].isin(rp9_accounting_ids)]

,ticketid,id
0,WAR230113TD03,0
1,WAR230113TD04,1
2,WAR230203TTXX,2
3,WAR230303TH01,3
4,WAR230318CC05,4
...,...,...
65,WAR241025HD09,65
66,WAR241025HD16,66
67,WAR241025HD17,67
68,WAR241025HDXX,68
